Pour ce 3e exercice il sera question d'implementer les algortithmes min-apriori, apriori, FP-Growth et les appliquer ceux-ci sur le jeu de donnees **Online_Retail** qui est une BD transactionnelle  qui contient les transactions d'une entreprise de vente au détail basée au Royaume-Uni.Cette base nouus l'avons eu sur le site UCI Repository.

In [7]:
import pandas as pd
import numpy as np
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

try:
    df=pd.read_excel('../data/OnlineRetail.xlsx')
except FileNotFoundError:
    print("Le jeu de donnees n'est pas charge")

print("Apercu des donnees brutes:")
display(df.head())
print(f"\nDimensions du jeu de donnees: {df.shape}")

Apercu des donnees brutes:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom



Dimensions du jeu de donnees: (541909, 8)


## 1. Prétraitement des Données

Le jeu de données est une liste de produits vendus. Nous devons le transformer en une liste de **transactions**, où chaque transaction est identifiée par son `InvoiceNo`.

Le processus de nettoyage inclut :

- Supprimer les transactions annulées (celles dont `InvoiceNo` commence par 'C').

- Supprimer les lignes avec des descriptions de produits vides (`Description` NaN).

- Convertir le tout en une liste de listes, où chaque sous-liste représente un panier.

In [16]:
#Nettoyage

df.dropna(axis=0, subset=['InvoiceNo'], inplace=True) # Suppression des lignes sans numero de factures

df['InvoiceNo']= df['InvoiceNo'].astype('str')

df=df[~df['InvoiceNo'].str.contains('C')] # Suppression des transactions annulees

df.dropna(axis=0, subset=['Description'], inplace=True) #Suppression des lignes sans description

df['Description']=df['Description'].str.strip() #Nettoyage des descriptions

# Regroupement des produits par facture pour creer des transactions(On le fait sur un seul pays la france)
basket_fr=(df[df['Country']== "France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

#On ne conserve que les produits; on ne s'occupe pas de la qte
def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

#On conserve les transactions avec au moins 2 articles
basket_encoded= basket_fr.map(encode_units)
basket_fr_filtered=basket_encoded[(basket_encoded > 0).sum(axis=1) >= 2]

transactions_list=[]
for i in range(len(basket_fr_filtered)):
    transaction=basket_fr_filtered.columns[basket_fr_filtered.iloc[i,:].to_numpy().nonzero()].tolist()
    transactions_list.append(transaction)

print(f"Nombre de transactions pour la France apres filtrage: {len(transactions_list)}")
print("\n Exemple de 2 transactions (paniers)")
print(transactions_list[:2])

Nombre de transactions pour la France apres filtrage: 376

 Exemple de 2 transactions (paniers)
[['ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED', 'CHARLOTTE BAG DOLLY GIRL DESIGN', 'CIRCUS PARADE LUNCH BOX', 'INFLATABLE POLITICAL GLOBE', 'LUNCH BOX I LOVE LONDON', 'MINI JIGSAW CIRCUS PARADE', 'MINI JIGSAW SPACEBOY', 'MINI PAINT SET VINTAGE', 'PANDA AND BUNNIES STICKER SHEET', 'POSTAGE', 'RED TOADSTOOL LED NIGHT LIGHT', 'ROUND SNACK BOXES SET OF4 WOODLAND', 'SET 2 TEA TOWELS I LOVE LONDON', 'SET/2 RED RETROSPOT TEA TOWELS', 'SPACEBOY LUNCH BOX', 'STARS GIFT TAPE', 'VINTAGE HEADS AND TAILS CARD GAME', 'VINTAGE SEASIDE JIGSAW PUZZLES'], ['CHARLOTTE BAG DOLLY GIRL DESIGN', 'MINI JIGSAW DOLLY GIRL', 'MINI JIGSAW SPACEBOY', 'PICTURE DOMINOES', 'POLKADOT RAIN HAT', 'POSTAGE', 'VINTAGE HEADS AND TAILS CARD GAME']]


In [22]:
##2. Algorithme FP-Growth et Apriori

#la librairie mlxtend attend les donnees binarisees(Il faut donc appliquer du one-hot encodinng)

te=TransactionEncoder()
te_ary=te.fit(transactions_list).transform(transactions_list)
df_onehot=pd.DataFrame(te_ary, columns=te.columns_)

min_sup_mlxtend=0.1 # on fixe a 10%


start_time_ap= time.time()
frequent_itemsets_ap=apriori(df_onehot, min_support=min_sup_mlxtend, use_colnames=True)
end_time_ap=time.time()
print(f"Temps d'execution de Apriori: {end_time_ap - start_time_ap:.4f} secondes")


start_time_fp= time.time()
frequent_itemsets_fp=fpgrowth(df_onehot, min_support=min_sup_mlxtend, use_colnames=True)
end_time_fp=time.time()
print(f"Temps d'execution de FP-Growth: {end_time_fp - start_time_fp:.4f} secondes")

#Juste les 10 premiers
print("\n 10 Itemsets Fréquents trouvés ")
display(frequent_itemsets_fp.sort_values(by='support', ascending=False).head(10))

# Génération des règles d'association à partir des résultats de FP-Growth
rules = association_rules(frequent_itemsets_fp, metric="lift", min_threshold=1)
print("\n--- Top 10 Règles d'Association (triées par lift) ---")
display(rules.sort_values(by='lift', ascending=False).head(10))

Temps d'execution de Apriori: 0.0531 secondes
Temps d'execution de FP-Growth: 0.3297 secondes

 10 Itemsets Fréquents trouvés 


,support,itemsets
0,0.784574,(POSTAGE)
26,0.196809,(RABBIT NIGHT LIGHT)
1,0.188830,(RED TOADSTOOL LED NIGHT LIGHT)
11,0.178191,(PLASTERS IN TIN WOODLAND ANIMALS)
20,0.175532,(PLASTERS IN TIN CIRCUS PARADE)
50,0.172872,"(RABBIT NIGHT LIGHT, POSTAGE)"
2,0.164894,(ROUND SNACK BOXES SET OF4 WOODLAND)
27,0.164894,"(POSTAGE, RED TOADSTOOL LED NIGHT LIGHT)"
7,0.159574,(LUNCH BAG RED RETROSPOT)
28,0.154255,"(ROUND SNACK BOXES SET OF4 WOODLAND, POSTAGE)"



--- Top 10 Règles d'Association (triées par lift) ---


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
40,(SET/6 RED SPOTTY PAPER PLATES),"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",0.132979,0.106383,0.103723,0.780000,7.332000,1.0,0.089577,4.061896,0.996067,0.764706,0.753810,0.877500
37,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.106383,0.132979,0.103723,0.975000,7.332000,1.0,0.089577,34.680851,0.966422,0.764706,0.971166,0.877500
39,(SET/6 RED SPOTTY PAPER CUPS),"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",0.143617,0.106383,0.103723,0.722222,6.788889,1.0,0.088445,3.217021,0.995700,0.709091,0.689153,0.848611
38,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.106383,0.143617,0.103723,0.975000,6.788889,1.0,0.088445,34.255319,0.954212,0.709091,0.970807,0.848611
25,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.132979,0.143617,0.127660,0.960000,6.684444,1.0,0.108562,21.409574,0.980828,0.857143,0.953292,0.924444
24,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.143617,0.132979,0.127660,0.888889,6.684444,1.0,0.108562,7.803191,0.993012,0.857143,0.871847,0.924444
33,(SET/6 RED SPOTTY PAPER CUPS),"(POSTAGE, SET/6 RED SPOTTY PAPER PLATES)",0.143617,0.111702,0.106383,0.740741,6.631393,1.0,0.090341,3.426292,0.991615,0.714286,0.708139,0.846561
32,"(POSTAGE, SET/6 RED SPOTTY PAPER PLATES)",(SET/6 RED SPOTTY PAPER CUPS),0.111702,0.143617,0.106383,0.952381,6.631393,1.0,0.090341,17.984043,0.955988,0.714286,0.944395,0.846561
30,"(SET/6 RED SPOTTY PAPER CUPS, POSTAGE)",(SET/6 RED SPOTTY PAPER PLATES),0.122340,0.132979,0.106383,0.869565,6.539130,1.0,0.090114,6.647163,0.965152,0.714286,0.849560,0.834783
35,(SET/6 RED SPOTTY PAPER PLATES),"(SET/6 RED SPOTTY PAPER CUPS, POSTAGE)",0.132979,0.122340,0.106383,0.800000,6.539130,1.0,0.090114,4.388298,0.976994,0.714286,0.772121,0.834783


On constate avec le temps d'execution que Apriori est plus rapide que FP-

 - La Nature du Jeu de Données : Un Dataset Sparse
Le jeu de données "Online Retail" a des milliers d'articles uniques. Cependant, chaque transaction (panier) ne contient qu'un très petit sous-ensemble de ces articles.

- L'élagage d'Apriori est très efficace : Comme la plupart des articles uniques sont rares, l'algorithme Apriori n'a pas besoin de générer beaucoup de candidats de taille 2, 3, etc. Le principe d'Apriori ("si {A} est rare, alors {A, B} est forcément rare") fonctionne à plein régime et élague l'arbre de recherche de manière drastique et très tôt. Le coût de la génération de candidats reste donc faible.

- Le coût de construction de l'arbre de FP-Growth devient dominant : Pour FP-Growth, même avec un min_support élevé, il doit d'abord scanner toute la base de données deux fois pour construire l'arbre FP-Tree en mémoire. Pour des données "sparse" où les transactions sont très différentes les unes des autres, la compression offerte par l'arbre n'est pas optimale. Le coût initial de construction de cet arbre complexe peut alors devenir plus élevé que le coût total d'un Apriori très efficace.
